In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install ultralytics
!pip install torch torchvision


In [ ]:
%cat /content/drive/MyDrive/Final_project/image_detect/dataset/data.yaml

Yolov8 가져오기

In [ ]:
# %cd /content/drive/MyDrive/Final_project/image_detect
# !git clone https://github.com/ultralytics/ultralytics

In [ ]:
#필요한 패키지 설치
# %cd /content/drive/MyDrive/Final_project/image_detect/ultralytics
# !pip install -e .


In [ ]:
#yolov8n.pt 파일이 생성됨
# !yolo predict

모델 학습

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
data_yaml = '/content/drive/MyDrive/Final_project/image_detect/dataset/data.yaml'
model.train(data=data_yaml, epochs=100, batch=16, imgsz=416, cache=True, name='team2_yolov8n_model' )

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/Final_project/image_detect/ultralytics/runs/detect/team2_yolov8n_model

모델 평가

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# 전이 학습된 YOLOv8 모델 로드 (best.pt 경로를 사용)
model = YOLO('/content/drive/MyDrive/Final_project/image_detect/ultralytics/runs/detect/team2_yolov8n_model/weights/best.pt')  # 모델 파일 경로를 입력하세요

# 평가할 이미지 로드
image_path = '/content/drive/MyDrive/Final_project/image_detect/dataset/my_face.jpg'
image = cv2.imread(image_path)

# 이미지가 None인지 확인
if image is None:
    raise ValueError(f"Image not found or unable to open: {image_path}")

# 이미지를 RGB로 변환
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 모델을 사용하여 예측
results = model.predict(image_rgb)

# 예측된 바운딩 박스를 이미지에 그리기
for result in results:
    for box in result.boxes.xyxy:
        x1, y1, x2, y2 = map(int, box[:4])  # 바운딩 박스 좌표
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (255, 0, 0), 2)  # 빨간색 바운딩 박스

# Matplotlib으로 이미지를 출력
plt.imshow(image_rgb)
plt.axis('off')
plt.show()


In [ ]:
print(results[0].boxes.xyxy)  # 바운딩 박스 좌표들 출력


In [ ]:
import os

label_dir = '/content/drive/MyDrive/Final_project/image_detect/dataset/train/labels'

# 레이블 파일들 확인
for label_file in os.listdir(label_dir):
    with open(os.path.join(label_dir, label_file), 'r') as file:
        for line in file:
            values = line.strip().split()
            if len(values) != 5:
                print(f"Error in file {label_file}: {line}")

1이 넘어가는 값이 있는지 확인

In [ ]:
import os

label_dir = '/content/drive/MyDrive/Final_project/image_detect/dataset/train/labels'

for label_file in os.listdir(label_dir):
    with open(os.path.join(label_dir, label_file), 'r') as file:
        for line in file:
            values = line.strip().split()
            class_id, x_center, y_center, width, height = map(float, values)
            if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and
                    0 <= width <= 1 and 0 <= height <= 1):
                print(f"Out of bounds in file {label_file}: {line}")
